In [1]:
import PyPDF2
import os
import pandas as pd
from langdetect import detect
import re
from collections import Counter

def get_pdf_files(directory):
    return [os.path.join(directory, file) for file in os.listdir(directory) if file.lower().endswith('.pdf')]

def extract_info_from_pdfs(pdf_paths):
    data = []
    
    # Espressione regolare per rilevare URL
    url_pattern = re.compile(r'https?://[^\s]+')
    
    for pdf_path in pdf_paths:
        try:
            num_pages = 0  # Numero totale di pagine
            word_count = 0  # Conteggio delle parole totali
            language = 'unknown'  # Lingua predefinita
            word_frequencies = Counter()  # Contatore per parole più frequenti
            link_count = 0  # Conteggio dei link
            file_size = os.path.getsize(pdf_path)  # Dimensione del file in byte

            with open(pdf_path, 'rb') as f:
                reader = PyPDF2.PdfReader(f)
                num_pages = len(reader.pages)  # Numero di pagine del PDF
                
                # Estrai metadati
                title = reader.metadata.get('/Title', 'Non disponibile')
                author = reader.metadata.get('/Author', 'Non disponibile')
                subject = reader.metadata.get('/Subject', 'Non disponibile')
                producer = reader.metadata.get('/Producer', 'Non disponibile')
                created = reader.metadata.get('/CreationDate', 'Non disponibile')
                
                # Rileva la lingua del PDF (basato sul testo della prima pagina per esempio)
                first_page_text = reader.pages[0].extract_text() if reader.pages else ''
                if first_page_text.strip():
                    try:
                        language = detect(first_page_text)
                    except:
                        language = 'unknown'

                # Conteggio delle parole, identificazione delle parole più frequenti e link
                for page in reader.pages:
                    text = page.extract_text() if page else ''
                    word_count += len(text.split())
                    
                    # Aggiungi parole alla frequenza (escludendo punteggiatura, e considerando solo parole di almeno 4 caratteri)
                    words = re.findall(r'\b\w{4,}\b', text.lower())  # \b\w{4,}\b corrisponde a parole di almeno 4 caratteri
                    word_frequencies.update(words)
                    
                    # Conta i link nel testo
                    links = re.findall(url_pattern, text)
                    link_count += len(links)

            # Parola più frequente
            most_common_word = word_frequencies.most_common(1)
            most_common_word = most_common_word[0][0] if most_common_word else 'Non disponibile'

            # Aggiungi i dati per ogni PDF (una sola volta per PDF)
            data.append({
                'pdf_name': os.path.basename(pdf_path),
                'total_pages': num_pages,
                'word_count': word_count,
                'language': language,
                'title': title,
                'author': author,
                'subject': subject,
                'producer': producer,
                'created': created,
                'most_common_word': most_common_word,
                'link_count': link_count,
                'file_size_bytes': file_size
            })
        except (PyPDF2.PdfReadError, FileNotFoundError, IOError) as e:
            print(f"Errore nella lettura del file '{pdf_path}': {e}")
        except Exception as e:
            print(f"Errore imprevisto nel file '{pdf_path}': {e}")

    return pd.DataFrame(data)


In [2]:
# Specifica la directory contenente i PDF
pdf_directory =  './pdfs'

# Trova tutti i file PDF nella directory
pdf_files = get_pdf_files(pdf_directory)

if not pdf_files:
    print("Nessun file PDF trovato nella directory specificata.")
else:
    # Estrazione delle informazioni dai PDF
    pdf_info_df = extract_info_from_pdfs(pdf_files)

    if pdf_info_df.empty:
        print("Non è stato possibile estrarre alcuna informazione dai PDF.")
    else:
        # Mostra le prime righe del DataFrame risultante
        display(pdf_info_df.head())


,pdf_name,total_pages,word_count,language,title,author,subject,producer,created,most_common_word,link_count,file_size_bytes
0,sample-1.pdf,2,321,ca,Brochure,Non disponibile,Non disponibile,Skia/PDF m111 Google Docs Renderer,Non disponibile,dolor,0,69988
1,sample-2.pdf,1,142,ca,Letter,Non disponibile,Non disponibile,Skia/PDF m111 Google Docs Renderer,Non disponibile,your,0,22698
2,sample-3.pdf,1,313,ca,Resume,Non disponibile,Non disponibile,Skia/PDF m111 Google Docs Renderer,Non disponibile,month,0,40622
